In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as pl
from pandas_datareader.data import DataReader
from statsmodels import regression
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

In [ ]:
def linear_regression(x,y):
    x = sm.add_constant(x)
    model = regression.linear_model.OLS(y,x).fit()
    x = x[:, 1]
    return model.params[0], model.params[1]

In [ ]:
BTC = DataReader('BTC-USD',  'yahoo', '2015-01-01', '2018-12-01')
DOW = DataReader('^DJI',  'yahoo', '2015-01-01', '2018-12-01')
BTC_return_monthly = BTC.groupby(pd.Grouper(freq='MS'))['Adj Close'].mean()
BTC_return_monthly = BTC_return_monthly.pct_change()[1:]
DOW_return_monthly = DOW.groupby(pd.Grouper(freq='MS'))['Adj Close'].mean()
DOW_return_monthly = DOW_return_monthly.pct_change()[1:]
DOW_return_monthly.plot(label = "DJIA")
BTC_return_monthly.plot(label = "SPY")
pl.ylabel("Monthly Return of Dow-Jones and BTC")
pl.legend(loc="upper left")
pl.show()

In [ ]:
BTC_return_monthly = BTC_return_monthly[:46]
if BTC_return_monthly.shape == DOW_return_monthly.shape:
    pass
else:
    raise exception("Shapes don't match.")

In [ ]:
BTC_return_monthly

In [ ]:
X = DOW_return_monthly.values
Y = BTC_return_monthly.values
Alpha, Beta = linear_regression(X,Y)
X2 = np.linspace(X.min(), X.max(), 100)
Y_tilda = X2* Beta + Alpha
pl.scatter(X,Y, alpha = 0.3)
pl.xlabel("Dow-Jones Monthly return")
pl.ylabel("BTC Monthly return")
pl.plot(X2, Y_tilda, 'r', alpha = 0.9)
pl.show()
Slope, Intercept, R_Value, P_Value, Std_Err = scipy.stats.linregress(X, Y)
print("Alpha = " + str(Alpha) + "\n")
print("R_Value = " +  str(R_Value) + "\n")
print("Slope = " +   str(Slope) + "\n")
print("Std_Err = " +  str(Std_Err) + "\n")

In [ ]:
dataframe = pd.concat([BTC_return_monthly, DOW_return_monthly], axis=1)

In [ ]:
dataframe.columns = ['BTC-Monthly', 'Dow-Monthly']

In [ ]:
sns.pairplot(dataframe, x_vars=['Dow-Monthly'], y_vars='BTC-Monthly', height = 4.5, aspect = 2.0, kind='reg')

In [ ]:
import pymc3 as pm
lin_model = pm.Model()

In [ ]:
with lin_model:
    intercept = pm.Normal('intercept',mu=0, sd = 20)
    beta = pm.Normal('beta',0, sd = 20)
    sigma = pm.HalfCauchy('sigma', beta = 10)
    likelihood = pm.Normal('Y', mu = intercept + beta * DOW_return_monthly, sd = sigma, observed = BTC_return_monthly)
    trace = pm.sample(3000)

In [ ]:
pl.figure(figsize=(7, 7))
pm.traceplot(trace[100:])
pl.tight_layout();

In [ ]:
pm.plot_posterior(trace)

In [ ]:
import numpy as np
n_samples = 500
x_range = np.linspace(DOW_return_monthly.min(),DOW_return_monthly.max())


for i in np.arange(n_samples,step=10):
    yhat = trace['intercept'][i] + trace['beta'][i] * x_range
    pl.plot(x_range,yhat,color='k',alpha = 0.2)
    pl.scatter(DOW_return_monthly,BTC_return_monthly,color='r',marker='+', label = "data")
    pl.ylabel('BTC monthly return')
    pl.xlabel('DOW-Jones monthly return')

In [ ]:
diverging = trace['diverging']
print('Number of Divergent Chains: {}'.format(diverging.nonzero()[0].size))
diverging_perc = diverging.nonzero()[0].size / len(trace) * 100
print('Percentage of Divergent Chains: {:.1f}'.format(diverging_perc))

In [ ]:
pm.forestplot(trace)

In [ ]:
pm.summary(trace)

In [ ]:
BTC_return_monthly.shape

In [ ]:
BTC

In [ ]:
pm.summary(trace)['Rhat']

In [ ]:
pm.stats.waic(trace, model= lin_model, pointwise= False, progressbar=False)